# Driver History Tables

In [1]:
import pandas as pd
import copy
import warnings
import datetime as dt
import numpy as np

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [2]:
# Import all the data
drivers_df = pd.read_csv("./f1db_csv/drivers.csv").drop(columns = "url")
results_df = pd.read_csv("./f1db_csv/results.csv")
constructors_df = pd.read_csv("./f1db_csv/constructors.csv")
races_df = pd.read_csv("./f1db_csv/races.csv")
qualifying_df = pd.read_csv("./f1db_csv/qualifying.csv")
d_standings_df = pd.read_csv("./f1db_csv/driver_standings.csv")

# Clean some names and create new variables
# drivers_df
drivers_df["number"] = drivers_df["number"].replace({r"\N": None})
drivers_df["driverName"] = drivers_df["forename"].str.cat(drivers_df["surname"],sep = " ")
drivers_df = drivers_df.drop(columns = ["forename", "surname"])

# results_df
results_df["position"] = results_df["position"].replace({r"\N": None})

# Merging to a full dataset for driver history
df_1 = pd.merge(drivers_df[["driverId", "driverName", "number", "nationality"]], results_df[["driverId", "raceId", "constructorId", "position", "fastestLapTime"]], on = "driverId")
df_2 = pd.merge(df_1, constructors_df[["constructorId", "name"]], on = "constructorId")
df_2 = df_2.rename({"position" : "racePosition"}, axis = "columns")
df_2 = df_2.rename({"name" : "constructorName"}, axis = "columns")
df_3 = pd.merge(df_2, d_standings_df[["driverId", "raceId", "points", "position", "wins"]], on = ["driverId", "raceId"])
df_3 = df_3.rename({"position" : "driverStanding"}, axis = "columns")
df_4 = pd.merge(df_3, races_df[["raceId", "year", "name", "date"]], on = "raceId")
df_5 = pd.merge(df_4, qualifying_df[["raceId", "driverId", "position", "q1", "q2", "q3"]], on = ["driverId", "raceId"])

for i in range(len(df_5['q1'])):
    nan_series = df_5.q1.isna()[i]
    if (df_5['q1'][i] == r"\N") | (nan_series == True):
        df_5['q1'][i] = None
        i += 1
    elif df_5['q1'][i] != 0:
        df_5['q1'][i] = float(str(df_5['q1'][i]).split(':')[1]) + (60 * float(str(df_5['q1'][i]).split(':')[0]))
        i += 1
    else:
        df_5['q1'][i] = None
        i += 1

for i in range(len(df_5['q2'])):
    nan_series = df_5.q2.isna()[i]
    if (df_5['q2'][i] == r"\N") | (nan_series == True):
        df_5['q2'][i] = None
        i += 1
    elif df_5['q2'][i] != 0:
        df_5['q2'][i] = float(str(df_5['q2'][i]).split(':')[1]) + (60 * float(str(df_5['q2'][i]).split(':')[0]))
        i += 1
    else:
        df_5['q2'][i] = None
        i += 1

for i in range(len(df_5['q3'])):
    nan_series = df_5.q3.isna()[i]
    if (df_5['q3'][i] == r"\N") | (nan_series == True):
        df_5['q3'][i] = None
        i += 1
    elif df_5['q3'][i] != 0:
        df_5['q3'][i] = float(str(df_5['q3'][i]).split(':')[1]) + (60 * float(str(df_5['q3'][i]).split(':')[0]))
        i += 1
    else:
        df_5['q3'][i] = None
        i += 1
        
for i in range(len(df_5['fastestLapTime'])):
    nan_series = df_5.fastestLapTime.isna()[i]
    if (df_5['fastestLapTime'][i] == r"\N") | (nan_series == True):
        df_5['fastestLapTime'][i] = None
        i += 1
    elif df_5['fastestLapTime'][i] != 0:
        df_5['fastestLapTime'][i] = float(str(df_5['fastestLapTime'][i]).split(':')[1]) + (60 * float(str(df_5['fastestLapTime'][i]).split(':')[0]))
        i += 1
    else:
        df_5['fastestLapTime'][i] = None
        i += 1

df_5["minQualifyingTime"] = df_5[["q1", "q2", "q3"]].min(skipna = True, axis = 1)
df_5 = df_5.drop(columns = ["q1", "q2", "q3"])

In [3]:
df_minlap = pd.read_csv("./f1db_csv/min_laps.csv")
df_5 = df_5.merge(df_minlap,on='raceId')
df_5[df_5.year == 2020]

,driverId,driverName,number,nationality,raceId,constructorId,racePosition,fastestLapTime,constructorName,points,driverStanding,wins,year,name,date,position,minQualifyingTime,Unnamed: 0,minOverallRaceLap,minOverallQualiLap
7350,832,Carlos Sainz,55,Spanish,1031,1,5,67.974,McLaren,10.0,5,0,2020,Austrian Grand Prix,2020-07-05,8,63.971,355,67.475,62.939
7351,846,Lando Norris,4,British,1031,1,3,67.475,McLaren,16.0,3,0,2020,Austrian Grand Prix,2020-07-05,4,63.626,355,67.475,62.939
7352,1,Lewis Hamilton,44,British,1031,131,4,67.712,Mercedes,12.0,4,0,2020,Austrian Grand Prix,2020-07-05,2,62.951,355,67.475,62.939
7353,822,Valtteri Bottas,77,Finnish,1031,131,1,67.657,Mercedes,25.0,1,1,2020,Austrian Grand Prix,2020-07-05,1,62.939,355,67.475,62.939
7354,847,George Russell,63,British,1031,3,None,69.317,Williams,0.0,15,0,2020,Austrian Grand Prix,2020-07-05,17,65.167,355,67.475,62.939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7685,850,Pietro Fittipaldi,51,Brazilian,1047,210,19,101.707,Haas F1 Team,0.0,23,0,2020,Abu Dhabi Grand Prix,2020-12-13,19,98.173,371,100.926,95.246
7686,842,Pierre Gasly,10,French,1047,213,8,102.474,AlphaTauri,75.0,10,1,2020,Abu Dhabi Grand Prix,2020-12-13,10,96.242,371,100.926,95.246
7687,826,Daniil Kvyat,26,Russian,1047,213,11,102.704,AlphaTauri,32.0,14,0,2020,Abu Dhabi Grand Prix,2020-12-13,7,95.963,371,100.926,95.246
7688,815,Sergio Pérez,11,Mexican,1047,211,None,103.263,Racing Point,125.0,4,1,2020,Abu Dhabi Grand Prix,2020-12-13,15,96.034,371,100.926,95.246


In [4]:
df_5['race_lap_ratio'] = df_5['fastestLapTime']/df_5['minOverallRaceLap']
df_5['quali_lap_ratio'] = df_5['minQualifyingTime']/df_5['minOverallQualiLap']
# Turn date into datetime
df_5["date"] = pd.to_datetime(df_5["date"])

In [5]:
# Clean this dataset: drop variables and rearrange
df = copy.deepcopy(df_5)
df = df.drop(columns = ["driverId", "constructorId"])
df = df[["driverName", "number", "nationality", "year", "name", "date","raceId", "constructorName", "position", "minQualifyingTime", "racePosition", "fastestLapTime", "wins", "points", "driverStanding"]]

# Turn date into datetime
df["date"] = pd.to_datetime(df["date"])

# Save it into a csv
df.to_csv("./f1db_csv/driver_history.csv")

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7602 entries, 0 to 7601
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   driverName         7602 non-null   object        
 1   number             4475 non-null   object        
 2   nationality        7602 non-null   object        
 3   year               7602 non-null   int64         
 4   name               7602 non-null   object        
 5   date               7602 non-null   datetime64[ns]
 6   raceId             7602 non-null   int64         
 7   constructorName    7602 non-null   object        
 8   position           7602 non-null   int64         
 9   minQualifyingTime  7498 non-null   float64       
 10  racePosition       5994 non-null   object        
 11  fastestLapTime     6395 non-null   object        
 12  wins               7602 non-null   int64         
 13  points             7602 non-null   float64       
 14  driverSt

In [7]:
# Function that chooses race history for a particular driver
def driver_history_selection(driver_name):
    driver_table = df[df.driverName == driver_name]
    driver_table = driver_table.sort_values("date", ascending = False)
    return driver_table

driver_history_selection("Alexander Albon")

,driverName,number,nationality,year,name,date,raceId,constructorName,position,minQualifyingTime,racePosition,fastestLapTime,wins,points,driverStanding
7483,Alexander Albon,23,Thai,2020,Tuscan Grand Prix,2020-09-13,1039,Red Bull,4,75.914,3,80.039,0,63.0,5
7463,Alexander Albon,23,Thai,2020,Italian Grand Prix,2020-09-06,1038,Red Bull,9,80.064,15,84.926,0,48.0,6
7443,Alexander Albon,23,Thai,2020,Belgian Grand Prix,2020-08-30,1037,Red Bull,5,102.193,6,108.736,0,48.0,4
7423,Alexander Albon,23,Thai,2020,Spanish Grand Prix,2020-08-16,1036,Red Bull,6,77.029,8,82.194,0,40.0,6
7403,Alexander Albon,23,Thai,2020,70th Anniversary Grand Prix,2020-08-09,1035,Red Bull,9,86.642,5,89.477,0,36.0,6
7383,Alexander Albon,23,Thai,2020,British Grand Prix,2020-08-02,1034,Red Bull,12,86.545,8,88.689,0,26.0,6
7363,Alexander Albon,23,Thai,2020,Hungarian Grand Prix,2020-07-19,1033,Red Bull,13,75.715,5,79.44,0,22.0,5
7343,Alexander Albon,23,Thai,2020,Styrian Grand Prix,2020-07-12,1032,Red Bull,7,79.014,4,67.299,0,12.0,8
7323,Alexander Albon,23,Thai,2020,Austrian Grand Prix,2020-07-05,1031,Red Bull,5,63.746,13,68.432,0,0.0,13
7305,Alexander Albon,23,Thai,2019,Abu Dhabi Grand Prix,2019-12-01,1030,Red Bull,6,95.682,6,102.219,0,92.0,8


In [8]:
import plotly.express as px
df_grouped = [y for x,y in df_5.groupby('driverName',as_index=False)]
for i in range(len(df_grouped)):
    df_driver_grouped = [y for x,y in df_grouped[i].groupby('year',as_index=False)]
    for j in range(len(df_driver_grouped)):
        #df_driver_grouped[j].drop(['minOverallRaceLap','minOverallQualiLap','race_lap_ratio','number'],inplace=True,axis=1)
        df_driver_grouped[j] = df_driver_grouped[j][['driverName','date','quali_lap_ratio']]
        df_driver_grouped[j].dropna(inplace=True)
        df_driver_grouped[j] = df_driver_grouped[j].sort_values(by='date')
        #print(df_driver_grouped[j]['date'])
        df_driver_grouped[j] = df_driver_grouped[j].reset_index(drop=True)
        
        if df_driver_grouped[j].empty:
            continue
        
        last_row = copy.deepcopy(df_driver_grouped[j].tail(1))
        curr_year = last_row.iloc[0]['date'].year
        first_day = dt.datetime(curr_year,1,1)
        
        days_in_year = []
        for k in range(len(df_driver_grouped[j])):
            date = df_driver_grouped[j].iloc[k].date
            days_in_year.append((date-first_day).days)
        df_driver_grouped[j]['days_in_year'] = days_in_year
        
        if len(df_driver_grouped[j])>1:
            fit = np.polyfit(x=df_driver_grouped[j]['days_in_year'],y=df_driver_grouped[j]['quali_lap_ratio'],deg=1)
            linear = np.poly1d(fit)
            df_driver_grouped[j]['fit_quali_lap_ratio'] = linear(df_driver_grouped[j]['days_in_year'])
            for l in range(len(df_driver_grouped[j]['fit_quali_lap_ratio'])):
                if df_driver_grouped[j]['fit_quali_lap_ratio'][l]<1:
                    df_driver_grouped[j]['fit_quali_lap_ratio'][l]=1.0
        elif len(df_driver_grouped[j]==1):
            df_driver_grouped[j]['fit_quali_lap_ratio'] = df_driver_grouped[j]['quali_lap_ratio'].iloc[0]
        
        #rolling avg
        #df_driver_grouped[j]['rolling_quali_lap_ratio']=df_driver_grouped[j]['quali_lap_ratio'].rolling(30,min_periods=1,win_type='boxcar',center=True).mean()
        
        
        last_row.date = dt.datetime(curr_year,12,31)
        last_row.rolling_quali_lap_ratio = None
        last_row.quali_lap_ratio = None
        last_row.fit_quali_lap_ratio = None
        df_driver_grouped[j] = df_driver_grouped[j].append(last_row)
    df_grouped[i] = pd.concat(df_driver_grouped)
df_6 = pd.concat(df_grouped).sort_values(by='date')
df_6[df_6.driverName=='Max Verstappen'].tail(50)

,driverName,date,quali_lap_ratio,days_in_year,fit_quali_lap_ratio
1,Max Verstappen,2018-04-08,1.0161,97.0,1.008112
2,Max Verstappen,2018-04-15,1.0077,104.0,1.008201
3,Max Verstappen,2018-04-29,1.00489,118.0,1.008379
4,Max Verstappen,2018-05-13,1.00844,132.0,1.008558
5,Max Verstappen,2018-06-10,1.00244,160.0,1.008915
6,Max Verstappen,2018-06-24,1.00751,174.0,1.009093
7,Max Verstappen,2018-07-01,1.01125,181.0,1.009183
8,Max Verstappen,2018-07-08,1.00827,188.0,1.009272
9,Max Verstappen,2018-07-22,1.00857,202.0,1.009450
10,Max Verstappen,2018-07-29,1.00357,209.0,1.009540


In [20]:
fig = px.line(df_6,x='date',y='quali_lap_ratio',color='driverName')
fig.show()

In [21]:
fig = px.line(df_6,x='date',y='fit_quali_lap_ratio',color='driverName')
fig.show()